# Eye-tracking : analysis

In [ ]:
import csv
#numpy and panda for data structure
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

%matplotlib inline
import cv2
import time
import itertools
from tslearn.metrics import dtw_path_from_metric
import seaborn as sns
from mpl_toolkits import mplot3d

from matplotlib._png import read_png
from matplotlib.cbook import get_sample_data

## Import data

Please refer to "Eye-tracking analysis - Extract fixations.ipynb" on how the files in folder "/fixations" should be generated from the raw data. 

In [ ]:
#unpacking fixation data
fix_original_all_d1, fix_original_all_d2, fix_original_all_d3, fix_original_all_d4, fix_original_all_d5, fix_original_all_d6 = [
    pd.read_csv("./fixations/dialog"+str(i)+"_fixations_42_original.csv", index_col=0) for i in range(1,7)]

In [ ]:
fixations_original_all = pd.concat([fix_original_all_d1, fix_original_all_d2, fix_original_all_d3, fix_original_all_d4, fix_original_all_d5, fix_original_all_d6])

In [ ]:
#fixations_original_all.head()

### Set the same id in each condition for the same participant

In [ ]:
#import ids of participants
path_original_all = "./participants/42_original.csv"
original_all = pd.read_csv(path_original_all)

In [ ]:
#original_all

In [ ]:
ids_original_all = original_all[['dialog1','dialog2','dialog3','dialog4','dialog5','dialog6']].values

In [ ]:
ids_original_all.shape

In [ ]:
for i in range(len(ids_original_all)):    
    for index in ids_original_all[i]:
#         fixations_original_all.loc[(fixations_original_all.tester_id == index), 'tester_id'] = i
        print(index)
        print(i)
#         print(fixations_original_all.tester_id)
        print("---------")
#         fixations_original_all.loc[(fixations_original_all.tester_id == index), 'tester_id_new'] = i

In [ ]:
len(ids_original_all)

In [ ]:
fixations_original_all.tester_id.unique()

tester_id is equal to PID in the surveys

In [ ]:
#fixations_original_all.loc[fixations_original_all.tester_id == 0]

In [ ]:
#fixations_original_all.to_csv('fixations_original_all.csv')

## Fixation duration

As a recall, the fixations were identified with a gaze velocity threshold of 21px/s, and a minimum fixation duration of 100ms on denoised data (interpolation for 60Hz and noise reduction level using median of 21 consecutives points).

see: https://support.realeye.io/fixation-filter/#:~:text=Fixation%20is%20a%20series%20of,terms%20of%20time%20and%20space.

### First, I compute the mean fixation duration (of the fixation points) for each participant in each condition.

In [ ]:
participant_mean_fix_duration_original_all = []
for part_id in range(len(fixations_original_all.tester_id.unique())):
    participant_values = []
    for item_name in fixations_original_all.index.unique():
        nb_fixation = fixations_original_all.loc[
            (fixations_original_all.index ==item_name) 
            & (fixations_original_all.tester_id == part_id)].shape[0]
        total_duration = np.sum(
            fixations_original_all.loc[
                (fixations_original_all.index ==item_name) 
                & (fixations_original_all.tester_id == part_id), 'fixation_duration_ms'].tolist())
        mean_dur = total_duration/nb_fixation
        participant_values.append(mean_dur)
    participant_mean_fix_duration_original_all.append(participant_values)
participant_mean_fix_duration_original_all = pd.DataFrame(participant_mean_fix_duration_original_all, columns=fixations_original_all.index.unique().tolist())

In [ ]:
participant_mean_fix_duration_original_all

In [ ]:
#participant_mean_fix_duration_original_all.to_csv('participant_mean_fix_duration_original_all.csv', index=False)

### Then I compute the mean fixation duration per video (real analysis on SPSS)

In [ ]:
mean_fix_duration_original_all = {}
for dialog in participant_mean_fix_duration_original_all.keys():
    mean_fix_duration_original_all[dialog]=np.mean(participant_mean_fix_duration_original_all[dialog].values)

In [ ]:
mean_fix_duration_original_all

*** SPSS reference: https://www.statisticssolutions.com/the-wilcoxon-sign-test-in-spss/#:~:text=The%20Wilcoxon%20sign%20test%20is,or%20with%20ranked%2Fordinal%20data.

## Total fixation duration (ratio with duration of each video)

### First, I compute the total fixation duration for each participant in each condition.

In [ ]:
dialog_durations = []
# dialog_durations.append(fixations_original_all.index.unique().tolist())
dialog_name = ['dialog1_normal_mask','dialog2_head_mask','dialog3_spatial','dialog4_head','dialog5_spatial_mask','dialog6_normal']
dialog_durations.append(dialog_name)
dialog_durations.append([143800,116400,130240,104200,124160,135200])
dialog_durations = pd.DataFrame(dialog_durations, columns=dialog_durations.pop(0))

In [ ]:
fixations_original_all.index.unique().tolist()

In [ ]:
dialog_durations

In [ ]:
participant_total_fix_duration_original_all = []
for part_id in range(len(fixations_original_all.tester_id.unique())):
    participant_values = []
    for item_name in fixations_original_all.index.unique():
        total_duration = np.sum(
            fixations_original_all.loc[
                (fixations_original_all.index ==item_name) 
                & (fixations_original_all.tester_id == part_id), 'fixation_duration_ms'].tolist())
        ratio = total_duration / dialog_durations[item_name].values.item()
        participant_values.append(ratio)
    participant_total_fix_duration_original_all.append(participant_values)
participant_total_fix_duration_original_all = pd.DataFrame(participant_total_fix_duration_original_all, columns=fixations_original_all.index.unique().tolist())

In [ ]:
fixations_original_all.tester_id.unique()

In [ ]:
fixations_original_all.index.unique()

In [ ]:
#participant_total_fix_duration_original_all.to_csv('participant_total_fix_duration_original_all.csv', index=False)

### Then I compute the mean total fixation duration based on these values (real analysis on SPSS)

In [ ]:
mean_total_fix_duration_original_all = {}
for dialog in participant_total_fix_duration_original_all.keys():
    mean_total_fix_duration_original_all[dialog]=np.mean(participant_total_fix_duration_original_all[dialog].values)

In [ ]:
mean_total_fix_duration_original_all

## Fixation number ratio with duration of the videos

In [ ]:
participant_nb_fixation = []
for part_id in range(len(fixations_original_all.tester_id.unique())):
    participant_values = []
    for item_name in fixations_original_all.index.unique():
        nb_fixation = fixations_original_all.loc[
                (fixations_original_all.index ==item_name) 
                & (fixations_original_all.tester_id == part_id)].shape[0]
        ratio = nb_fixation / dialog_durations[item_name].values.item()
        participant_values.append(ratio)
    participant_nb_fixation.append(participant_values)
participant_nb_fixation = pd.DataFrame(participant_nb_fixation, columns=fixations_original_all.index.unique().tolist())

In [ ]:
#participant_nb_fixation.to_csv('nb_fixation.csv',index=False)

In [ ]:
mean_nb_fix = {}
for dialog in participant_nb_fixation.keys():
    mean_nb_fix[dialog]=np.mean(participant_nb_fixation[dialog].values)

In [ ]:
mean_nb_fix

## Mean saccades amplitude per participant

The saccade amplitude is the distance between two successive fixation points produced by one participant.
Thus, to compute the saccade amplitudes I calculate the euclidean distance between all successive fixation points. The euclidean distance is also the L2 distance, thus I use the numpy.linalg.norm function to calculate the euclidean distance for better performance.

In [ ]:
#fixations_original_all

In [ ]:
list_points = fixations_original_all.loc[(fixations_original_all.tester_id==0) & (fixations_original_all.index == 'dialog1_normal_mask'),"fixation_point_x":"fixation_point_y"].values.tolist()

In [ ]:
list_dist = []
for index, elem in enumerate(list_points):
    if (index+1 < len(list_points)):
        point_start = np.array(elem)
        point_end = np.array(list_points[index+1])
        dist = np.linalg.norm(point_end-point_start)
    list_dist.append(dist)
mean_saccade_amplitude = np.mean(list_dist)
print(mean_saccade_amplitude)

In [ ]:
participant_saccade_amplitude_original_all = []
for part_id in range(len(fixations_original_all.tester_id.unique())):
    participant_values = []
    for item_name in fixations_original_all.index.unique():
        list_points = fixations_original_all.loc[
            (fixations_original_all.tester_id==part_id) & (fixations_original_all.index == item_name),
            "fixation_point_x":"fixation_point_y"].values.tolist()
        list_dist = []
        for index, elem in enumerate(list_points):
            if (index+1 < len(list_points)):
                point_start = np.array(elem)
                point_end = np.array(list_points[index+1])
                dist = np.linalg.norm(point_end-point_start)
            list_dist.append(dist)
        mean_saccade_amplitude = np.mean(list_dist)
        participant_values.append(mean_saccade_amplitude)
    participant_saccade_amplitude_original_all.append(participant_values)
participant_saccade_amplitude_original_all = pd.DataFrame(participant_saccade_amplitude_original_all, columns=fixations_original_all.index.unique().tolist())

In [ ]:
#participant_saccade_amplitude_original_all.to_csv('participant_mean_saccade_amplitude_original_all.csv', index=False)

### Then I compute the mean saccade amplitude for each condition with the above values (real analysis on SPSS)

In [ ]:
mean_saccade_amplitude_original_all = {}
for dialog in participant_saccade_amplitude_original_all.keys():
    mean_saccade_amplitude_original_all[dialog]=np.mean(participant_saccade_amplitude_original_all[dialog].values)

In [ ]:
mean_saccade_amplitude_original_all

# Temporal analysis (frame by frame)

## Dispersion (Fixations points variability per frame)

The dispersion is the mean of the Euclidian distances between multiple observers points. In the following, the focus is put on the dispersion of fixation points for each frame of each condition.

All the videos have a frame rate of 25 frame/s. Thus, for each frame, we calculate the mean of the euclidean distances over all the fixation points on the frame.

### First I need to regroup all the fixation points per frame for each dialog

Each frame correspond to a duration of 40 ms. Thus, to have the fixation points for each frame, I regroup all the fixation points that appear on it in the interval of time of the frame (0-40, 41-80, 81-120, ...)

In [ ]:
#fixations_original_all.head()

In [ ]:
#dialog_durations

In [ ]:
fixations_original_all.index.unique()

In [ ]:
#comment the next line to execute this cell
# %%script false --no-raise-error
# structure : 
# first level = dialog1, dialog2, ...
# second level = frame 1, frame 2, ... 
# third level = fixation point 1, fixation point 2, ..
frame_time = round(1000 / 25)
fixation_points_per_frame = []
for item_name in fixations_original_all.index.unique(): # 6 dialogs
    list_points_dialog = []
    for frame_nb in range(0, round(dialog_durations[[item_name]].values.item() / frame_time)): # each frame
        frame_begin = frame_nb * frame_time
        frame_end = (frame_nb+1) * frame_time
        list_points_frame = fixations_original_all.loc[
                (fixations_original_all.index == item_name)
                & (
                    (
                        (fixations_original_all.fixation_starts_at_ms >= frame_begin) 
                        & (fixations_original_all.fixation_starts_at_ms < frame_end)
                        & (fixations_original_all.fixation_ends_at_ms > frame_end)
                    ) |
                    (
                        (fixations_original_all.fixation_starts_at_ms <= frame_begin)
                        & (fixations_original_all.fixation_ends_at_ms > frame_begin )
                        & (fixations_original_all.fixation_ends_at_ms < frame_end )
                    ) |
                    (
                        (fixations_original_all.fixation_starts_at_ms >= frame_begin)
                        & (fixations_original_all.fixation_starts_at_ms < frame_end)
                        & (fixations_original_all.fixation_ends_at_ms > frame_begin)
                        & (fixations_original_all.fixation_ends_at_ms < frame_end)
                    ) |
                        (
                            (fixations_original_all.fixation_starts_at_ms <= frame_begin)
                            & (fixations_original_all.fixation_ends_at_ms >= frame_end)
                        )
                  ),
                "fixation_point_x":"fixation_point_y"].values.tolist() # set of points
        list_points_dialog.append(list_points_frame)
    fixation_points_per_frame.append(list_points_dialog)

In [ ]:
%store fixation_points_per_frame

In [ ]:
%store -r fixation_points_per_frame

In [ ]:
fixation_points_per_frame = []
fixation_dialog = []
fixation_frame = []
count_dialog = 1

with open('./stored_variables/fixations_points_per_frame.txt') as f:
    
    for line in f.readlines():
        count_br = line.count("[")
        line = line.replace("[", "")
        line = line.replace("]", "")
        line = line.replace("\n", "")
        line = line.replace(" ", "")
        
        fixation_value = []
        for i in line.split(','):
            if i != '':
                fixation_value.append(int(i))
        if (count_br > 3) or (count_br == 1):
            if (count_br > 3):
                print(line)
            fixation_frame.append(fixation_value)
        elif count_br == 2:
            fixation_dialog.append(fixation_frame)
            fixation_frame = []
            fixation_frame.append(fixation_value)
        else: #count_br == 3
            fixation_points_per_frame.append(fixation_dialog)
            fixation_dialog = []
            count_dialog = count_dialog + 1
            fixation_frame = []
            fixation_frame.append(fixation_value)
    fixation_points_per_frame.append(fixation_dialog)

In [ ]:
fixation_frame

In [ ]:
count_dialog

### Fixation points per participant per frame (used later for scanpath vizualisation in 3D)

In [ ]:
fixations_original_all.tester_id.unique()

In [ ]:
#comment the next line to execute this cell
# %%script false --no-raise-error
# structure : 
# first level = dialog1, dialog2, ...
# second level = participant1, participant2, ...
# third level = frame 1, frame 2, ... 
# fourth level = fixation point 1, fixation point 2, ..
frame_time = round(1000 / 25)
fixation_points_per_participant_per_frame = []
for item_name in fixations_original_all.index.unique(): # 6 dialogs
    list_points_dialog = []
    for tester_id in fixations_original_all.tester_id.unique(): # each pariticpant
        list_points_participant = []
        for frame_nb in range(0, round(dialog_durations[[item_name]].values.item() / frame_time)): # each frame
            frame_begin = frame_nb * frame_time
            frame_end = (frame_nb+1) * frame_time
            list_points_frame = fixations_original_all.loc[
                    (fixations_original_all.index == item_name)
                    & (fixations_original_all.tester_id == tester_id)
                    & (
                        (
                            (fixations_original_all.fixation_starts_at_ms >= frame_begin) 
                            & (fixations_original_all.fixation_starts_at_ms < frame_end)
                            & (fixations_original_all.fixation_ends_at_ms > frame_end)
                        ) |
                        (
                            (fixations_original_all.fixation_starts_at_ms <= frame_begin)
                            & (fixations_original_all.fixation_ends_at_ms > frame_begin )
                            & (fixations_original_all.fixation_ends_at_ms < frame_end )
                        ) |
                        (
                            (fixations_original_all.fixation_starts_at_ms >= frame_begin)
                            & (fixations_original_all.fixation_starts_at_ms < frame_end)
                            & (fixations_original_all.fixation_ends_at_ms > frame_begin)
                            & (fixations_original_all.fixation_ends_at_ms < frame_end)
                        ) |
                        (
                            (fixations_original_all.fixation_starts_at_ms <= frame_begin)
                            & (fixations_original_all.fixation_ends_at_ms >= frame_end)
                        )
                      ),
                    "fixation_point_x":"fixation_point_y"].values.tolist()
            list_points_participant.append(list_points_frame)
        list_points_dialog.append(list_points_participant)
    fixation_points_per_participant_per_frame.append(list_points_dialog)

In [ ]:
fixations_original_all.tester_id.unique()

In [ ]:
%store fixation_points_per_participant_per_frame

In [ ]:
%store -r fixation_points_per_participant_per_frame

In [ ]:
fixation_points_per_participant_per_frame = pd.read_pickle(r'fixation_points_per_participant_per_frame.pkl')

In [ ]:
# fixation_points_per_participant_per_frame = []
# fixation_dialog = []
# fixation_participant = []
# fixation_frame = []
# count_dialog = 1

# with open('./stored_variables/fixation_points_per_participant_per_frame.txt') as f:
    
#     for line in f.readlines():
#         count_br = line.count("[")
#         line = line.replace("[", "")
#         line = line.replace("]", "")
#         line = line.replace("\n", "")
#         line = line.replace(" ", "")
        
#         fixation_value = []
#         for i in line.split(','):
#             if i != '':
#                 fixation_value.append(int(i))
#         if (count_br > 4) or (count_br == 1):
#             if (count_br > 4):
#                 print(line)
#             fixation_frame.append(fixation_value)
#         elif count_br >= 1:
#             fixation_dialog.append(fixation_frame)
#             fixation_frame = []
#             fixation_frame.append(fixation_value)
#         elif count_br == 3:
#             fixation_dialog.append(fixation_participant)
#             fixation_participant = []
#             fixation_frame = []
#             fixation_frame.append(fixation_value)
#         else:  #count_br == 4
#             fixation_points_per_participant_per_frame.append(fixation_dialog)
#             fixation_dialog = []
#             fixation_participant = []
#             fixation_frame = []
#             fixation_frame.append(fixation_value)
#     fixation_points_per_participant_per_frame.append(fixation_dialog)

In [ ]:
fixation_points_per_participant_per_frame = pd.read_pickle(r'fixation_points_per_participant_per_frame.pkl')

### Now I can compute the dispersion for each frame in each dialog

In [ ]:
dialog_nb = 0
dispersion_per_frame = []
for dialog in fixation_points_per_frame:
    dialog_nb = dialog_nb+1
    print("Current analysis of dispersion in dialog "+str(dialog_nb))
    dispersion_dialog = []
    for frame in dialog:
        if len(frame)>1:
            for i in frame:
                dispersion_values = []
                for j in frame:
                    if i!=j:
                        point_i = np.array(i)
                        point_j = np.array(j)
                        dist = np.linalg.norm(point_j-point_i) #operate 41 times
                        dispersion_values.append(dist)
            dispersion = np.sum(dispersion_values)/((len(frame)-1)*len(frame))
            dispersion_dialog.append(dispersion)
        else:
            dispersion_dialog.append(0)
    dispersion_per_frame.append(dispersion_dialog)

In [ ]:
# dispersion_per_frame.to_pickle('dispersion_per_frame.pkl')

In [ ]:
len(dispersion_per_frame[5])

In [ ]:
dispersion_per_frame[5][-5:]

Export dispersion per frame to csv for each condition

In [ ]:
# dialog_nb = 0
# for item_name in fixations_original_all.index.unique():
#     df = pd.DataFrame(dispersion_per_frame[dialog_nb],columns=[item_name])
#     df.to_csv(item_name+'_dispersion_per_frame_original_all.csv', index=False)
#     dialog_nb = dialog_nb+1

In [ ]:
fixations_original_all.index.unique()

In [ ]:
# load dispersion per from
dispersion_per_frame = []
for item_name in fixations_original_all.index.unique():
    dispersion_per_frame.append(pd.read_csv('./results/dispersions_per_frame/original/' + item_name+'_dispersion_per_frame_original_all.csv'))

In [ ]:
dispersion_per_frame

### Now I plot the dispersion over time for each dialog

Replace missing dispersion value with previous frame value

In [ ]:
for dialog in dispersion_per_frame:
    if 0 in dialog:
        dialog_array = np.array(dialog)
        ids_zero = np.where(dialog_array == 0)[0]
        for id_zero in ids_zero:
            if id_zero != 0:
                dialog[id_zero] = dialog[id_zero-1]

Create points where x = start time in ms of a frame and y = its corresponding dispersion value

In [ ]:
frame_time = round(1000 / 25)
dialog_nb = 0
list_points_dialog_disp = []
for item_name in fixations_original_all.index.unique():
    list_points = []
    for frame_nb in range(0, round(dialog_durations[[item_name]].values.item() / frame_time)):
        frame_begin = frame_nb * frame_time
        try:
#             disp_value = dispersion_per_frame[dialog_nb][frame_nb]
            disp_value = dispersion_per_frame[dialog_nb].loc[frame_nb].values[0]
            point = [frame_begin, disp_value]
            list_points.append(point)
        except: #modified, the original one does not have problems
            print("dialog: " + str(dialog_nb) + " ;frame: " + str(frame_nb) + " value is not valid")
#             print(dispersion_per_frame[dialog_nb][frame_nb])

    list_points_dialog_disp.append(list_points)
    dialog_nb = dialog_nb+1

In [ ]:
frame_nb

In [ ]:
dialog_nb

In [ ]:
dispersion_per_frame[0].loc[0].values[0]

In [ ]:
#list_points_dialog_disp[0][0]

Now I make the plots

In [ ]:
frame_start = 1
frame_end = 200

selected_dialogs = [3,4,6]
plt.figure(figsize=(12,8))
plt.title('Dispersion over time for auditive conditions without mask (frame '+str(frame_start)+' to '+str(frame_end)+')')
for i in selected_dialogs:
    x = [point[0] for point in list_points_dialog_disp[i-1][frame_start:frame_end]]
    y = [point[1] for point in list_points_dialog_disp[i-1][frame_start:frame_end]]
    if i == 6:
        plt.plot(x, y, label='mono', marker='o', color='c')
    elif i == 4:
        plt.plot(x, y, label='binaural-with-head-rotations', marker='x', color='r')
    elif i == 3:
        plt.plot(x, y, label='binaural', marker='^', color='g')
plt.xlabel('Time in ms')
plt.ylabel('Dispersion')
plt.legend()
plt.show()
#plt.savefig('dispersion.png')

selected_dialogs = [1,2,5]
plt.figure(figsize=(12,8))
plt.title('Dispersion over time for auditive conditions with mask (frame '+str(frame_start)+' to '+str(frame_end)+')')
for i in selected_dialogs:
    x = [point[0] for point in list_points_dialog_disp[i-1][frame_start:frame_end]]
    y = [point[1] for point in list_points_dialog_disp[i-1][frame_start:frame_end]]
    if i == 1:
        plt.plot(x, y, label='mono_mask', marker='o', color='c')
    elif i == 2:
        plt.plot(x, y, label='head_mask', marker='x', color='r')
    elif i == 5:
        plt.plot(x, y, label='spatial_mask', marker='^', color='g')
plt.xlabel('Time in ms')
plt.ylabel('Dispersion')
plt.legend()
plt.show()
#plt.savefig('dispersion.png')

### Mean dispersion per condition (real analysis on SPSS)

In [ ]:
mean_dispersion = {}
dialog_nb = 0
for item_name in fixations_original_all.index.unique():
    mean_dispersion[item_name] = np.mean(dispersion_per_frame[dialog_nb])
    dialog_nb = dialog_nb+1

In [ ]:
mean_dispersion

In [ ]:
mean_dispersion

## Distance to center

Here I measure the distance of the fixations to the center for each frame. To do so, I use the fixation points of each frame and compute their centroid. Then I compute the distance of the centroid to the center of the image (640, 360)

In [ ]:
#np.array(fixation_points_per_frame[0][0]).shape[0]

In [ ]:
dist_center_per_frame = []
centroid_per_frame = []
center = np.array([640,360])
dialog_nb = 0
for item_name in fixations_original_all.index.unique():
    dist_values = []
    centroid_dialog = []
    for frame in fixation_points_per_frame[dialog_nb]:
        arr = np.array(frame)
        length = arr.shape[0]
        if length > 1:
            sum_x = np.sum(arr[:, 0])
            sum_y = np.sum(arr[:, 1])
            centroid = np.array([sum_x/length, sum_y/length])
            dist = np.linalg.norm(centroid-center)
            dist_values.append(dist)
            centroid_dialog.append(centroid)
        else:
            dist_values.append(0)
            centroid_dialog.append(np.array([640,360]))
    dist_center_per_frame.append(dist_values)
    centroid_per_frame.append(centroid_dialog)
    dialog_nb = dialog_nb+1

In [ ]:
#dist_center_per_frame

I export this to csv per condition

In [ ]:
#dialog_nb = 0
#for item_name in fixations_original_all.index.unique():
#    df = pd.DataFrame(dist_center_per_frame[dialog_nb],columns=[item_name])
#    df.to_csv(item_name+'_distance_center_per_frame_original_all.csv', index=False)
#    dialog_nb = dialog_nb+1

### Then I plot the distances

Replace missing distance value with previous frame value

In [ ]:
#for dialog in dist_center_per_frame:
#    if 0 in dialog:
#        dialog_array = np.array(dialog)
#        ids_zero = np.where(dialog_array == 0)[0]
#        for id_zero in ids_zero:
#            if id_zero != 0:
#                dialog[id_zero] = dialog[id_zero-1]

Create points where x = start time in ms of a frame and y = its corresponding distance to center value

In [ ]:
frame_time = round(1000 / 25)
dialog_nb = 0
list_points_dialog_center = []
for item_name in fixations_original_all.index.unique():
    list_points = []
    for frame_nb in range(0, round(dialog_durations[[item_name]].values.item() / frame_time)):
        frame_begin = frame_nb * frame_time
        dist_value = dist_center_per_frame[dialog_nb][frame_nb]
        point = [frame_begin, dist_value]
        list_points.append(point)
    list_points_dialog_center.append(list_points)
    dialog_nb = dialog_nb+1

In [ ]:
list_points_dialog_center[0][0]

Now I make the plots

In [ ]:
frame_start = 1
frame_end = 2000

selected_dialogs = [3,4,6]
plt.figure(figsize=(12,8))
plt.title('Distance to center over time for auditive conditions without mask (frame '+str(frame_start)+' to '+str(frame_end)+')')
for i in selected_dialogs:
    x = [point[0] for point in list_points_dialog_center[i-1][frame_start:frame_end]]
    y = [point[1] for point in list_points_dialog_center[i-1][frame_start:frame_end]]
    if i == 6:
        plt.plot(x, y, label='mono', marker='o', color='c')
    elif i == 4:
        plt.plot(x, y, label='head', marker='x', color='r')
    elif i == 3:
        plt.plot(x, y, label='spatial', marker='^', color='g')
plt.xlabel('Time in ms')
plt.ylabel('Distance to center')
plt.legend()
plt.show()
#plt.savefig('dispersion.png')

selected_dialogs = [1,2,5]
plt.figure(figsize=(12,8))
plt.title('Distance to center over time for auditive conditions with mask (frame '+str(frame_start)+' to '+str(frame_end)+')')
for i in selected_dialogs:
    x = [point[0] for point in list_points_dialog_center[i-1][frame_start:frame_end]]
    y = [point[1] for point in list_points_dialog_center[i-1][frame_start:frame_end]]
    if i == 1:
        plt.plot(x, y, label='mono_mask', marker='o', color='c')
    elif i == 2:
        plt.plot(x, y, label='head_mask', marker='x', color='r')
    elif i == 5:
        plt.plot(x, y, label='spatial_mask', marker='^', color='g')
plt.xlabel('Time in ms')
plt.ylabel('Distance to center')
plt.legend()
plt.show()
#plt.savefig('dispersion.png')

### Mean distance to center per condition (real analysis on SPSS)

In [ ]:
mean_center_dist = {}
dialog_nb = 0
for item_name in fixations_original_all.index.unique():
    mean_center_dist[item_name] = np.mean(dist_center_per_frame[dialog_nb])
    dialog_nb = dialog_nb+1

In [ ]:
mean_center_dist

# Areas of interest and Scanpaths

## Set AoI

Our data, obtained with standard webcam, are not of high quality. Thus when the participants look at a specific location, the measured gaze data might have some offsets from this location (even if a calibration was made beforehand). Actually a study made by the provider of the eye-tracking solution we used resulted in an average spatial accuracy of their system of 113px. This means that if I want to set AoIs I have to take this into account and I can't draw a precise area. Instead, I draw a subjective global area around the body of the actors and I name them: L,F,R (L for left actor, R for right actor and F for front actor). Let's draw the areas (the areas will be the same for every dialog)

In [ ]:
#unpacking screenshot of dialogs
list_screenshots = [
    cv2.imread("./screenshot/dialog"+str(i)+"_screenshot.png") for i in range(1,7)]
list_screenshots = [
    cv2.cvtColor(list_screenshots[i],cv2.COLOR_BGR2RGB) for i in range(0,6)
]
dialog1_screenshot, dialog2_screenshot, dialog3_screenshot, dialog4_screenshot, dialog5_screenshot, dialog6_screenshot = [
    np.copy(list_screenshots[i]) for i in range(0,6)
]

In [ ]:
L= [(125,313),(477,681)]
R= [(906,303),(1226,683)]
F= [(559,179),(830,444)]
color_red = (255,0,0) #red color
color_orange = (235, 155, 52)
color_green = (87, 163, 75)
thick = 10

In [ ]:
i=1
plt.box(False)
for image in list_screenshots:
   
    plt.figure(figsize=(12.8, 7.2), dpi=100)
    
    plt.title("Dialog "+str(i))
    screenshot = np.copy(image)
    screenshot = cv2.rectangle(screenshot,L[0],L[1],color_red, thick)
    screenshot = cv2.rectangle(screenshot,R[0],R[1],color_green, thick)
    screenshot = cv2.rectangle(screenshot,F[0],F[1],color_orange, thick)
    
    plt.imshow(screenshot)
    
    fig.savefig("./screenshot/aoi_screenshot/aoi_dialog"+str(i)+".png", bbox_inches='tight')

#     plt.savefig("./screenshot/aoi_screenshot/aoi_dialog"+str(i)+".png", bbox_inches='tight')
    i = i+1

## Vizualisation of distance to center

In [ ]:
center = [640,360]
for i, image in enumerate(list_screenshots):
    plt.figure(figsize=(12,8))
    plt.title("Dialog "+str(i+1))
    screenshot = np.copy(image)
    list_dialogs= dialog_durations.columns.tolist()
    dialog_name = list_dialogs[i]
    for frame_nb in range(0, round(dialog_durations[[dialog_name]].values.item() / frame_time)):
        point = centroid_per_frame[i][frame_nb]
        x = [point[0],center[0]]
        y = [point[1],center[1]]
        plt.plot(x,y)
    plt.imshow(screenshot)
    #plt.show()
    #plt.savefig('dist_center_dialog_'+str(i+1)+'.png')

## Vizualisation of centroid distribution

In [ ]:
center = [640,360]
for i, image in enumerate(list_screenshots):
    plt.figure(figsize=(12,8))
    plt.title("Dialog "+str(i+1))
    screenshot = np.copy(image)
    list_dialogs= dialog_durations.columns.tolist()
    dialog_name = list_dialogs[i]
    x = []
    y = []
    for frame_nb in range(0, round(dialog_durations[[dialog_name]].values.item() / frame_time)):
        point = centroid_per_frame[i][frame_nb]
        x.append(point[0])
        y.append(point[1])
    d = {'x':x, 'y':y}
    pdxy = pd.DataFrame(d)
    #plt.scatter(x,y)
    #plt.hist2d(x, y, bins=50, cmap=plt.cm.jet, cmin=1, alpha=0.5)
    hmax= sns.kdeplot(pdxy['x'], pdxy['y'], cmap="Reds", shade=True, alpha=0.7)
    hmax.collections[0].set_alpha(0)
    plt.imshow(screenshot,origin='upper')
    #plt.show()
    #plt.savefig('centroid_distribution_dialog_'+str(i+1)+'.png')

## Vizualisation of space-time cube (fixations points)

In [ ]:
for dialog_nb, dialog in enumerate(fixation_points_per_frame):
    x = []
    y = []
    z = []
    for frame_nb, frame in enumerate(dialog):
        if(len(frame)>0):
            for point in frame:
                x.append(point[0])
                y.append(point[1])
                z.append(frame_nb)
    fig = plt.figure(figsize = (12, 8))
    ax = plt.axes(projection ="3d")
    ax.set_xlim(1280,0)
    ax.set_ylim(0,720)
    ax.set_zlim(0,max(z))
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("frames")
    ax.scatter3D(np.array(x), np.array(y), np.array(z), color = "red")
    plt.title("3D representation of fixations points in dialog "+str(dialog_nb+1))

## Vizualisation of space-time cube (scanpath, unit=frame, no generalization on AOI)

In [ ]:
for dialog_nb, dialog in enumerate(fixation_points_per_participant_per_frame):
    fig = plt.figure(figsize = (12, 8))
    ax = plt.axes(projection ="3d")
    plt.title("3D representation of scanpath in dialog "+str(dialog_nb+1))
    #for i in range (0,2):
    participant = dialog[1]
    x = []
    y = []
    z = []
    for frame_nb, frame in enumerate(participant):
        if(len(frame)>0):
            for point in frame:
                x.append(point[0])
                y.append(point[1])
                z.append(frame_nb)
    ax.set_xlim(1280,0)
    ax.set_ylim(0,720)
    ax.set_zlim(0,max(z))
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("frames")
    ax.plot3D(np.array(x), np.array(y), np.array(z))

In [ ]:
len(dialog[1])

In [ ]:
empty_frame_count = 0
for frame_nb, frame in enumerate(participant):
    if not frame:
        empty_frame_count = empty_frame_count + 1
print(empty_frame_count)

In [ ]:
# import matplotlib.image as mpimg

# import numpy as np
# im = plt.imshow(np.flipud(plt.imread('./screenshot/dialog5_screenshot.png')), origin='lower')
# plt.show()
# image = mpimg.imread('./screenshot/dialog5_screenshot.png')
# plt.imshow(image)  
image = list_screenshots[4]
plt.figure(figsize=(12,8))
plt.title("Dialog "+str(5))
screenshot = np.copy(image)
screenshot = cv2.rectangle(screenshot,L[0],L[1],color_red, thick)
screenshot = cv2.rectangle(screenshot,R[0],R[1],color_green, thick)
screenshot = cv2.rectangle(screenshot,F[0],F[1],color_orange, thick)
plt.imshow(screenshot)

In [ ]:
from matplotlib._png import read_png
from matplotlib.cbook import get_sample_data

In [ ]:
# demo period

fig = plt.figure(figsize = (12, 8))
ax = fig.gca(projection='3d')

dialog_nb = 4
dialog = fixation_points_per_participant_per_frame[dialog_nb]

frame_start = 2051
frame_end = 2177

# ax = plt.axes(projection ="3d")
plt.title("3D representation of scanpath of each partcipant in dialog "+str(dialog_nb+1) + 
          " of frame:" + str(frame_start) + " to " + str(frame_end))

# plot image
img = read_png('./screenshot/dialog1_screenshot.png')
x, y = np.ogrid[0:img.shape[0], 0:img.shape[1]]
ax.plot_surface(x, y, np.atleast_2d(frame_start), rstride=8, cstride=8, facecolors=img, alpha=0.2)


# plot scanpath
for i in range (0,42): # loop through 42 participantw
    participant = dialog[i]
    x = []
    y = []
    z = []

    for frame_nb in range(frame_start-1, frame_end):
        if participant[frame_nb] != []:
            x.append(participant[frame_nb][0][1])
            y.append(participant[frame_nb][0][0])
            z.append(frame_nb)

    ax.set_xlim(0,720)
    ax.set_ylim(0,1280)
    ax.set_zlim(frame_start,frame_end)

    ax.set_xlabel("y")
    ax.set_ylabel("x")
    ax.set_zlabel("frames")
    
    ax.plot3D(np.array(x), np.array(y), np.array(z), c='royalblue')
    
ax.view_init(elev=25, azim=-15)


In [ ]:
areas_of_interest = ['L','R','F','e']
aoi_ratio_normal_transitions = {}
dialog_nb = 4
aoi_ratio_normal_transitions[dialog_nb] = {}

frame_start = 2109 - 25
frame_end = 2119 + 25
if frame_start < 0:
    frame_start = 0
if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
    frame_end = len(list_points_dialog_disp[dialog_nb])-1
plt.figure(figsize=(20,4))
plt.title('Fixation ratios on AoIs over frames for dialog '+str(dialog_nb+1)+' (window frames: '+str(frame_start)+' to '+str(frame_end)+') (transition frames: '+str(dialog_normal_transitions[dialog_nb][window_nb][0])+' to '+str(dialog_normal_transitions[dialog_nb][window_nb][1])+')'
         + '. Silence is marked in orange.' 
         )
for i in areas_of_interest:
    aoi_ratio_normal_transitions[dialog_nb][window_nb][i] = []
    x = []
    y = []
    for frame in range(frame_start-1, frame_end):
        x.append(frame)
        y.append(aoi_ratios_per_frame[list(aoi_ratios_per_frame.keys())[dialog_nb]][frame][i])
        aoi_ratio_normal_transitions[dialog_nb][window_nb][i].append(y)
    if i == 'L':
        plt.plot(x, y, label='left', marker='o', color='r')
    elif i == 'R':
        plt.plot(x, y, label='right', marker='x', color='g')
    elif i == 'F':
        plt.plot(x, y, label='front', marker='^', color='orange')
    else:
        plt.plot(x, y, label='exterior', marker='+', color='gray')

plt.axvspan(dialog_normal_transitions[dialog_nb][window_nb][0], dialog_normal_transitions[dialog_nb][window_nb][1], color='bisque', alpha=0.4) ## highlight silent area       

plt.xlabel('Frame')
plt.ylabel('Fixation ratio')
plt.legend()
plt.show()

In [ ]:
for dialog_nb, dialog in enumerate(fixation_points_per_participant_per_frame):
    fig = plt.figure(figsize = (12, 8))
    ax = plt.axes(projection ="3d")
    plt.title("3D representation of scanpath in dialog "+str(dialog_nb+1))
    for i in range (0,42):
        participant = dialog[i]
        x = []
        y = []
        z = []
        for frame_nb, frame in enumerate(participant):
            if(len(frame)>0):
                for point in frame:
                    x.append(point[0])
                    y.append(point[1])
                    z.append(frame_nb)
        ax.set_xlim(1280,0)
        ax.set_ylim(0,720)
        ax.set_zlim(0,max(z))
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("frames")
        ax.plot3D(np.array(x), np.array(y), np.array(z))

## Compute Scanpath

Here is a function to check if a point is in an area of interest (rect) or not.

In [ ]:
def pointInRect(point,rect):
    x1, y1, x2, y2 = rect
    x, y = point
    if (x1 <= x and x <= x2):
        if (y1 <= y and y <= y2):
            return True
    return False

I will now compute a scanpath for every participant in each condition. If the fixation point is in the AoI, I add the corresponding letter of the AoI to the scanpath, otherwise we add the character 'e' which means that the point is in none of the AoIs.

To do this, first I attribute an area of interest letter to every fixation point in our dataset.

In [ ]:
aoi_list = []

points_list = list(zip(fixations_original_all.fixation_point_x.tolist(), fixations_original_all.fixation_point_y.tolist()))
L_rect = [L[0][0], L[0][1], L[1][0], L[1][1]]
R_rect = [R[0][0], R[0][1], R[1][0], R[1][1]]
F_rect = [F[0][0], F[0][1], F[1][0], F[1][1]]

for point in points_list:
    if pointInRect(point, L_rect):
        aoi_list.append('L')
    elif pointInRect(point, R_rect):
        aoi_list.append('R')
    elif pointInRect(point, F_rect):
        aoi_list.append('F')
    else:
        aoi_list.append('e')
        
fixations_and_aoi = fixations_original_all.copy()
fixations_and_aoi['aoi'] = aoi_list

In [ ]:
fixations_and_aoi

In [ ]:
#fixations_and_aoi

In [ ]:
fixations_and_aoi.index.unique()

Then, I regroup all the characters together in a string for every participant in each dialog. (index is PID)

In [ ]:
participant_scanpaths = []
for part_id in range(len(fixations_and_aoi.tester_id.unique())):
    scanpaths = []
    for item_name in fixations_and_aoi.index.unique():
        string_path = ""
        string_path = string_path.join(fixations_and_aoi.loc[
            (fixations_and_aoi.index ==item_name) 
            & (fixations_and_aoi.tester_id == part_id), 'aoi'].tolist())
        scanpaths.append(string_path)
    participant_scanpaths.append(scanpaths)
participant_scanpaths = pd.DataFrame(participant_scanpaths, columns=fixations_original_all.index.unique().tolist())

In [ ]:
#participant_scanpaths.to_csv('long_scanpaths.csv',index=False)

In [ ]:
participant_scanpaths = pd.read_csv("./results/scanpaths/long_scanpaths.csv")

Create scanpath per participant per 

### Simplify individual scanpaths by abstracting consecutive repetitions (used later for calculating the number of transitions) (ratio with the number of speech acts in each videos)

In [ ]:
short_scanpaths = []
for i in range(0, 42):
    list_paths = []
    for column in participant_scanpaths:
        item = participant_scanpaths[column][i]
        path = ''.join(i for i, _ in itertools.groupby(item))
        list_paths.append(path)
    short_scanpaths.append(list_paths)
short_scanpaths = pd.DataFrame(short_scanpaths, columns=fixations_original_all.index.unique().tolist())

In [ ]:
#short_scanpaths.to_csv('short_scanpaths.csv',index=False)

### Compute Dissimilarity with Dynamic Time Warping (DTW)

I use the Dynamic Time Warping alogrithm (from tslearn) which aligns two sequences along a common time axis by considering only substitutions (the sequences do not need to be align on the same time points beforehands). With it I can compute the dissimilarity between two scanpaths. I use a sakoe chiba band of radius 1 and a predefined cost function (cost of 1 for all difference). It is not required that both time series share the same size, but they must be the same dimension.

For this I first need to transform my scanpaths into numerical values. I will therefore attribute the value 0 to 'e', 1 to 'R', 2 to 'F' and 3 to 'L'.

In [ ]:
for dialog in participant_scanpaths:
    print(dialog)
    for index, path in enumerate(participant_scanpaths[dialog]):
        print(index)
        print(path)

In [ ]:
num_scanpaths = {}
for dialog in participant_scanpaths:
    num_scanpaths[dialog] = {}
    for index, path in enumerate(participant_scanpaths[dialog]):
        num_path = [0 if element == 'e' else 1 if element == 'R' else 2 if element == 'F' else 3 if element == 'L' else element for element in path]
        num_scanpaths[dialog][index] = num_path

In [ ]:
num_path

I plot here a visualization of the scanpath between the different area of interest that I have set (the x-axis is simply the length of the scanpaths, it is not the actual duration of the fixation point)

In [ ]:
plt.figure(figsize=(20,4))
y = num_scanpaths['dialog3_spatial'][2]
x = [i for i,value in enumerate(y)]

plt.hlines(3, 0, len(x)-1, colors='red', linestyles='solid', label='Left')
plt.hlines(2, 0, len(x)-1, colors='orange', linestyles='solid', label='Front')
plt.hlines(1, 0, len(x)-1, colors='green', linestyles='solid', label='Right')
plt.hlines(0, 0, len(x)-1, colors='black', linestyles='solid', label='Exterior')

plt.plot(x,y)
plt.xlabel('Number of fixations')
plt.title('Visualization of the scanpath of participant 3 in dialog 3.')
plt.legend()
plt.show()

In [ ]:
def cost_fct(x,y):
    if x != y:
        return 1
    else:
        return 0

In [ ]:
#comment the next line to execute this cell
%%script false --no-raise-error
dissimilarity_results = {}
for dialog in num_scanpaths:
    dissimilarity_results[dialog] = {}
    for index1 in num_scanpaths[dialog]:
        dissimilarity_results[dialog][index1] = {}
        for index2 in num_scanpaths[dialog]:
            if index1 != index2:
                match, dissimilarity = dtw_path_from_metric(num_scanpaths[dialog][index1],
                                    num_scanpaths[dialog][index2],
                                    metric=cost_fct,
                                    global_constraint = "sakoe_chiba",
                                    sakoe_chiba_radius=1
                                   )
                dissimilarity_results[dialog][index1][index2] = dissimilarity

In [ ]:
#%store dissimilarity_results

In [ ]:
%store -r dissimilarity_results

Now I compute the mean dissimilarity of all participant between the all the other for each dialog. This will allow to then compute a mean dissimilarity over all participants and in the end indicate quantitatively how much the participants had a similar visual behaviour (low value indicate more similar behaviour).

In [ ]:
mean_dissimilarity = {}
for dialog in dissimilarity_results:
    mean_dissimilarity[dialog] = {}
    for part_id in dissimilarity_results[dialog]:
        mean_dissimilarity[dialog][part_id] = np.mean(list(dissimilarity_results[dialog][part_id].values()))

In [ ]:
mean_dissimilarity = pd.DataFrame(mean_dissimilarity)

In [ ]:
mean_dissimilarity_sorted = pd.DataFrame(np.sort(mean_dissimilarity.values, axis=0), index=mean_dissimilarity.index, columns=mean_dissimilarity.columns)

In [ ]:
#mean_dissimilarity.to_csv('mean_dissimilarity.csv', index=False)

In [ ]:
mean_dissimilarity_dialog = {}
for dialog in mean_dissimilarity:
    mean_dissimilarity_dialog[dialog]=np.mean(mean_dissimilarity[dialog].values)

In [ ]:
mean_dissimilarity_dialog

## AoIs Fixation ratio

For each frame, we can compute the ratio of each AoI (number of fixation points in the AoI divided by the number of fixation points in the frame). For this I first again regroup the fixation points by frame, with their corresponding AoI.

In [ ]:
#comment the next line to execute this cell
%%script false --no-raise-error
# structure : 
# first level = dialog1, dialog2, ...
# second level = frame 1, frame 2, ... 
# third level = fixation point 1, fixation point 2, ..
frame_time = round(1000 / 25)
aoi_per_frame = {}
for item_name in fixations_and_aoi.index.unique(): # each dialog
    aoi_per_frame[item_name] = {}
    for frame_nb in range(0, round(dialog_durations[[item_name]].values.item() / frame_time)): #each frame
        frame_begin = frame_nb * frame_time
        frame_end = (frame_nb+1) * frame_time
        list_points_frame = fixations_and_aoi.loc[
                (fixations_and_aoi.index == item_name)
                & (
                    (
                        (fixations_and_aoi.fixation_starts_at_ms >= frame_begin) 
                        & (fixations_and_aoi.fixation_starts_at_ms < frame_end)
                        & (fixations_and_aoi.fixation_ends_at_ms > frame_end)
                    ) |
                    (
                        (fixations_and_aoi.fixation_starts_at_ms <= frame_begin)
                        & (fixations_and_aoi.fixation_ends_at_ms > frame_begin )
                        & (fixations_and_aoi.fixation_ends_at_ms < frame_end )
                    ) |
                    (
                        (fixations_and_aoi.fixation_starts_at_ms >= frame_begin)
                        & (fixations_and_aoi.fixation_starts_at_ms < frame_end)
                        & (fixations_and_aoi.fixation_ends_at_ms > frame_begin)
                        & (fixations_and_aoi.fixation_ends_at_ms < frame_end)
                    ) |
                    (
                        (fixations_and_aoi.fixation_starts_at_ms <= frame_begin)
                        & (fixations_and_aoi.fixation_ends_at_ms >= frame_end)
                    ) 
                  ),
                "aoi"].values.tolist()
        aoi_per_frame[item_name][frame_nb] = list_points_frame

In [ ]:
#%store aoi_per_frame

In [ ]:
%store -r aoi_per_frame

In [ ]:
aoi_per_frame = pd.read_pickle(r'aoi_per_frame.pkl')

In [ ]:
import operator

aoi_ratios_per_frame = {}
for dialog in aoi_per_frame:
    aoi_ratios_per_frame[dialog] = {}
    for frame in aoi_per_frame[dialog]:
        aoi_ratios_per_frame[dialog][frame] = {}
        length = len(aoi_per_frame[dialog][frame])
        if length > 0 :
            aoi_ratios_per_frame[dialog][frame]['L'] = aoi_per_frame[dialog][frame].count('L')/length
            aoi_ratios_per_frame[dialog][frame]['R'] = aoi_per_frame[dialog][frame].count('R')/length
            aoi_ratios_per_frame[dialog][frame]['F'] = aoi_per_frame[dialog][frame].count('F')/length
            aoi_ratios_per_frame[dialog][frame]['e'] = aoi_per_frame[dialog][frame].count('e')/length
            # find the most frequent area
            dict_temp = dict((k, aoi_ratios_per_frame[dialog][frame][k]) for k in ('L', 'R', 'F', 'e'))
            dict_temp_note_e = dict((k, aoi_ratios_per_frame[dialog][frame][k]) for k in ('L', 'R', 'F'))
            
            aoi_ratios_per_frame[dialog][frame]['aoi_most'] = max(dict_temp.items(), key=operator.itemgetter(1))[0]
            aoi_ratios_per_frame[dialog][frame]['aoi_most_not_e'] = max(dict_temp_note_e.items(), key=operator.itemgetter(1))[0]
        else:
            aoi_ratios_per_frame[dialog][frame]['L'] = 0
            aoi_ratios_per_frame[dialog][frame]['R'] = 0
            aoi_ratios_per_frame[dialog][frame]['F'] = 0
            aoi_ratios_per_frame[dialog][frame]['e'] = 0

In [ ]:
aoi_ratios_per_frame['dialog1_normal_mask'][26][aoi_ratios_per_frame['dialog1_normal_mask'][26]['aoi_most']]

In [ ]:
aoi_ratios_per_frame['dialog1_normal_mask'][27]

In [ ]:
%store aoi_ratios_per_frame

In [ ]:
str(aoi_ratios_per_frame['dialog1_normal_mask'][1]['aoi_most_not_e'])

Plot the genetic scan with respect to frame (the area with the max no. of AOI is consider the AOI of the frame). 

In [ ]:
# include exterior (e)
num_scanpaths_aoi = {}
for dialog in aoi_ratios_per_frame:
    num_scanpaths_aoi[dialog] = {}
    num_path_aoi = []
    for frame in aoi_ratios_per_frame[dialog]:
        try:
            aoi_most_temp = str(aoi_ratios_per_frame[dialog][frame]['aoi_most'])
    #         print(aoi_most_temp)
    #         print('\n')
            if aoi_most_temp == 'e': 
                num_path_aoi.append(0)
            elif aoi_most_temp == 'R':
                num_path_aoi.append(1)
            elif aoi_most_temp == 'F':
                num_path_aoi.append(2)
            else: # 'L'
                num_path_aoi.append(3)
        except:
            num_path_aoi.append(None) # no AOI in this frame
            print('dialog: ' + dialog + ' frame:' + str(frame))
    num_scanpaths_aoi[dialog] = num_path_aoi

In [ ]:
# exclude exterior (e)
num_scanpaths_aoi_not_e = {}
for dialog in aoi_ratios_per_frame:
    num_scanpaths_aoi_not_e[dialog] = {}
    num_path_aoi_not_e = []
    for frame in aoi_ratios_per_frame[dialog]:
        try:
            aoi_most_temp_not_e = str(aoi_ratios_per_frame[dialog][frame]['aoi_most_not_e'])        
#             if aoi_most_temp == 'e': 
#                 num_path_aoi.append(0)
            if aoi_most_temp_not_e == 'R':
                num_path_aoi_not_e.append(1)
            elif aoi_most_temp_not_e == 'F':
                num_path_aoi_not_e.append(2)
            else: # 'L'
                num_path_aoi_not_e.append(3)
        except:
            num_path_aoi_not_e.append(None) # no AOI in this frame
            print('dialog: ' + dialog + ' frame:' + str(frame))
    num_scanpaths_aoi_not_e[dialog] = num_path_aoi_not_e

In [ ]:
len(y)

# 'dialog1_normal_mask': 122.24622531939605,
#  'dialog2_head_mask': 102.22299651567945,
#  'dialog3_spatial': 98.13356562137051,
#  'dialog4_head': 95.56329849012775,
#  'dialog5_spatial_mask': 94.78281068524971,
#  'dialog6_normal': 110.77700348432056

In [ ]:
# include exterior (e) 
for dialog in aoi_ratios_per_frame:

    plt.figure(figsize=(20,4))
    y = num_scanpaths_aoi[dialog]
    x = [i for i,value in enumerate(y)]

    plt.hlines(3, 0, len(x)-1, colors='red', linestyles='solid', label='Left')
    plt.hlines(2, 0, len(x)-1, colors='orange', linestyles='solid', label='Front')
    plt.hlines(1, 0, len(x)-1, colors='green', linestyles='solid', label='Right')
    plt.hlines(0, 0, len(x)-1, colors='black', linestyles='solid', label='Exterior')

    plt.plot(x,y)
    plt.xlabel('Frames')
    plt.title('Visualization of the genetic scanpath in '+ dialog)
    plt.legend()
    plt.show()

In [ ]:
# exclude exterior (e)
for dialog in aoi_ratios_per_frame:

    plt.figure(figsize=(20,4))
    y = num_scanpaths_aoi_not_e[dialog]
    x = [i for i,value in enumerate(y)]

    plt.hlines(3, 0, len(x)-1, colors='red', linestyles='solid', label='Left')
    plt.hlines(2, 0, len(x)-1, colors='orange', linestyles='solid', label='Front')
    plt.hlines(1, 0, len(x)-1, colors='green', linestyles='solid', label='Right')
#     plt.hlines(0, 0, len(x)-1, colors='black', linestyles='solid', label='Exterior')

    plt.plot(x,y)
    plt.xlabel('Frames')
    plt.title('Visualization of the genetic scanpath in '+ dialog + " excluding exterior")
    plt.legend()
    plt.show()

In [ ]:
# exclude exterior (e)
for dialog in aoi_ratios_per_frame:

    plt.figure(figsize=(20,4))
    y = num_scanpaths_aoi_not_e[dialog]
    x = [i for i,value in enumerate(y)]

    plt.hlines(3, 0, len(x)-1, colors='red', linestyles='solid', label='Left')
    plt.hlines(2, 0, len(x)-1, colors='orange', linestyles='solid', label='Front')
    plt.hlines(1, 0, len(x)-1, colors='green', linestyles='solid', label='Right')
#     plt.hlines(0, 0, len(x)-1, colors='black', linestyles='solid', label='Exterior')

    plt.plot(x,y)
    plt.xlabel('Frames')
    plt.title('Visualization of the genetic scanpath in '+ dialog + " excluding exterior")
    plt.legend()
    plt.show()

I now plot the evolution of the fixation ratio on the AoI over the frames:

In [ ]:
#aoi_ratios_per_frame['dialog1_normal_mask'][1]['L']

In [ ]:
dialog_durations

In [ ]:
areas_of_interest = ['L','R','F','e']
for dialog_nb in range(0,6):
    list_dialogs= dialog_durations.columns.tolist()
    dialog_name = list_dialogs[dialog_nb]
    frame_start = 1
    frame_end = int(dialog_durations.iloc[:, [dialog_nb]].values[0]/40)
    plt.figure(figsize=(20,4))
    plt.title('Fixation ratios on ROIs over frames in dialog '+str(dialog_nb+1)+' (frame '+str(frame_start)+' to '+str(frame_end)+')')
    for i in areas_of_interest:
        x = []
        y = []
        for frame in range(frame_start-1, frame_end):
            x.append(frame)
            y.append(aoi_ratios_per_frame[dialog_name][frame][i])
        if i == 'L':
            plt.scatter(x, y, label='left', marker='o', color='r')
        elif i == 'R':
            plt.scatter(x, y, label='right', marker='x', color='g')
        elif i == 'F':
            plt.scatter(x, y, label='front', marker='^', color='orange')
        else:
            plt.scatter(x, y, label='exterior', marker='+', color='black')
    plt.xlabel('Frames')
    plt.ylabel('Fixation ratio')
    plt.legend()
    #plt.savefig('aoi_ratio_dialog_'+str(dialog_nb+1)+'.png')
    plt.show()

In [ ]:
areas_of_interest = ['L','R','F','e']
for dialog_nb in range(0,6):
    list_dialogs= dialog_durations.columns.tolist()
    dialog_name = list_dialogs[dialog_nb]
    frame_start = 1
    frame_end = int(dialog_durations.iloc[:, [dialog_nb]].values[0]/40)
    plt.figure(figsize=(20,4))
    plt.title('Fixation ratios on ROIs over frames in dialog '+str(dialog_nb+1)+' (frame '+str(frame_start)+' to '+str(frame_end)+')')
    for i in areas_of_interest:
        x = []
        y = []
        for frame in range(frame_start-1, frame_end):
            x.append(frame)
            y.append(aoi_ratios_per_frame[dialog_name][frame][i])
        if i == 'L':
            plt.scatter(x, y, label='left', marker='o', color='r')
        elif i == 'R':
            plt.scatter(x, y, label='right', marker='x', color='g')
        elif i == 'F':
            plt.scatter(x, y, label='front', marker='^', color='orange')
        else:
            plt.scatter(x, y, label='exterior', marker='+', color='black')
    plt.xlabel('Frames')
    plt.ylabel('Fixation ratio')
    plt.legend()
    #plt.savefig('aoi_ratio_dialog_'+str(dialog_nb+1)+'.png')
    plt.show()

## (Global analysis) AoI number of fixations

In [ ]:
aoi_nb_fixation = {}
for item_name in fixations_and_aoi.index.unique():
    aoi_nb_fixation[item_name] = {}
    for area_of_interest in fixations_and_aoi.aoi.unique():
        nb_fixation = fixations_and_aoi.loc[
                (fixations_and_aoi.index == item_name) 
                & (fixations_and_aoi.aoi == area_of_interest)].shape[0]
        aoi_nb_fixation[item_name][area_of_interest] = nb_fixation

In [ ]:
#aoi_nb_fixation

## (Global analysis) AoI mean fixation duration

In [ ]:
aoi_fixation_dur = {}
for item_name in fixations_and_aoi.index.unique():
    aoi_fixation_dur[item_name] = {}
    for area_of_interest in fixations_and_aoi.aoi.unique():
        fix_dur = np.mean(fixations_and_aoi.loc[
                (fixations_and_aoi.index == item_name) 
                & (fixations_and_aoi.aoi == area_of_interest), 'fixation_duration_ms'].to_list())
        aoi_fixation_dur[item_name][area_of_interest] = fix_dur

In [ ]:
#aoi_fixation_dur

# Specific analysis (on group of frames / event)

In the following I will use some of the previous measure to make an analysis at particular time/event in the videos. First I import the data relative to the events in each video.

In [ ]:
dfs_event = [pd.read_csv('./data/event_timestamp/event_timestamp_dialog'+str(i)+'.csv', sep=';') for i in range(1,7)]

In [ ]:
#list_event = ['speaker_left','speaker_right','speaker_front']
#columns_name = ["speaker","event","frame_start","frame_end"]
#for i in range(0,6):
#    speaker_list = []
#    for event_name in list_event:
#        events = dfs_event[i].loc[(dfs_event[i].event == event_name+'_start'), "key_point"].to_list()
#        frame_start = dfs_event[i].loc[(dfs_event[i].event == event_name+'_start'), "frame_number"].to_list()
#        frame_end = dfs_event[i].loc[(dfs_event[i].event == event_name+'_stop'), "frame_number"].to_list()
#        for item_nb in range(0,len(events)):
#            speaker_list.append([event_name, events[item_nb], frame_start[item_nb], frame_end[item_nb]])
#    df = pd.DataFrame(speaker_list, columns=columns_name)
    #df.to_csv('dialog'+str(i+1)+'_speaker_events.csv', index=False)

In [ ]:
#dfs_speaker_event = [pd.read_csv('./data/event_timestamp/dialog'+str(i)+'_speaker_events.csv') for i in range(1,7)]

In [ ]:
#for i, df in enumerate(dfs_speaker_event):
#    dfs_speaker_event[i] = df.sort_values(by=['frame_start'])
#    dfs_speaker_event[i] = dfs_speaker_event[i].reset_index(drop=True)
#    dfs_speaker_event[i].to_csv('dialog'+str(i+1)+'_speaker_events.csv', index=False)

In [ ]:
dfs_speaker_event = [pd.read_csv('./data/event_timestamp/dialog'+str(i)+'_speaker_events.csv') for i in range(1,7)]

In [ ]:
# dfs_speaker_event[0].head()

In [ ]:
# dfs_speaker_event

## Visualization of the events (speaking actor, sound direction)

In [ ]:
color_direction = ['r','orange','g','grey'] #left, front, right, none
nb_speech_acts = {}
for i in range(0,6):
    plt.figure(figsize=(15,1))
    plt.title('Speech act in '+dialog_durations.keys().values[i])
    df = dfs_speaker_event[i] 
    last_frame = int(dialog_durations.iloc[:, [i]].values[0]/40)
    sound_df = dfs_event[i]
    sound_dir = sound_df[sound_df['event'].str.contains('sound')][["frame_number","event"]].values
    count_left = 0
    count_front = 0
    count_right = 0
    count_sound_changes = 0
    
    #plot sound direction - come from event_timestamp_dialogx (this is the binaural audio)
    if sound_dir[0][1] == 'no_sound_direction':
        y = 4
        plt.hlines(y,1,last_frame, colors=color_direction[3], linewidth=10, label='mono')
    else:
        for j in range(0,len(sound_dir)):
            direction = sound_dir[j][1]

            sound_start = sound_dir[j][0]
            if j < len(sound_dir)-1:
                next_sound_start = sound_dir[j+1][0]
            else:
                next_sound_start = last_frame
                
            if direction == 'sound_left':
                count_sound_changes += 1
                y = 4
                plt.hlines(y,sound_start,next_sound_start, colors=color_direction[0], linewidth=10)
            elif direction == 'sound_front':
                count_sound_changes += 1
                y = 4
                plt.hlines(y,sound_start,next_sound_start, colors=color_direction[1], linewidth=10)
            else:
                count_sound_changes += 1
                y = 4
                plt.hlines(y,sound_start,next_sound_start, colors=color_direction[2], linewidth=10)
    
    #plot act of speech (come from dialogx_speaker_events, this is the real act dfs_speaker_event)
    for j in range(0,len(df)):
        speaker = df.loc[(df.index == j), "speaker"].values[0]
        x_start = df.loc[(df.index == j), "frame_start"].values[0]
        x_end = df.loc[(df.index == j), "frame_end"].values[0]
        event = df.loc[(df.index == j), "event"].values[0]
        if speaker == 'speaker_left':
            count_left += 1
            y = 3
            plt.hlines(y,x_start,x_end, colors=color_direction[0], linewidth=10, label='left')
            #plt.text(y, x_start+(x_end-x_start)/2, event)
        elif speaker == 'speaker_front':
            count_front += 1
            y = 2
            plt.hlines(y,x_start,x_end, colors=color_direction[1], linewidth=10, label='front')
            #plt.text(y, x_start+(x_end-x_start)/2, event)
        else:
            count_right += 1
            y = 1
            plt.hlines(y,x_start,x_end, colors=color_direction[2], linewidth=10, label='right')
            #plt.text(y, x_start+(x_end-x_start)/2, event)
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys(),loc='lower left', bbox_to_anchor=(1, 0))
    plt.xlabel('Frames')
    #plt.savefig("speech_act_dialog"+str(i+1)+".pdf", bbox_inches='tight')
    plt.show()
    print('Nb of sound changes: '+str(count_sound_changes))
    print('Nb of speech act of actor on left: '+str(count_left))
    print('Nb of speech act of actor on front: '+str(count_front))
    print('Nb of speech act of actor on right: '+str(count_right))
    print('Total nb of speech act: '+str(count_right+count_left+count_front))
    nb_speech_acts[i] = count_right+count_left+count_front

We can see that there is a similar number of act of speech (minimum = 42 in dialog 5, maximum = 54 in dialog 6). The mean number of act of speech is 46.5. So the minimum is 4.5 away from the mean and the maximum is 7.5 away of the mean. It is also worth to note that an act of speech can be a "hmmm" or just "okay".

#### Ratio of number of transitions from one area of interest to another by the participants with the number of speech acts

In [ ]:
ratio_aoi_transitions = {}
for dialog_nb, dialog in enumerate(short_scanpaths.keys()):
    ratio_aoi_transitions[dialog] = []
    list_paths = short_scanpaths[dialog].tolist()
    for i in list_paths:
        ratio_aoi_transitions[dialog].append(len(i)/nb_speech_acts[dialog_nb])
ratio_aoi_transitions = pd.DataFrame(ratio_aoi_transitions, columns=short_scanpaths.keys())

In [ ]:
#ratio_aoi_transitions.to_csv('ratio_aoi_transitions.csv', index=False)

## Turn taking event

To identify turn taking events, I will first identify the transition between speakers when there is no overlap between them.
To do this I first merge the events occuring during other ones and then I check the gaps (or silences) between the events left which will be the transitions (without the starting silence and ending silence).

To merge overlapping intervals (ref: https://www.geeksforgeeks.org/merging-intervals/):
* 1) Sort all intervals in increasing order of start time.
* 2) Traverse sorted intervals starting from first interval, 
   do following for every interval.
       a) If current interval is not first interval and it 
         overlaps with previous interval, then merge it with
         previous interval. Keep doing it while the interval
         overlaps with the previous one.         
       b) Else add current interval to output list of intervals.

In [ ]:
# in O(n Log n) time and O(1) extra space 
def mergeIntervals(arr): 
          
        # Sorting based on the increasing order  
        # of the start intervals 
        arr.sort(key = lambda x: x[0])  
          
        # array to hold the merged intervals 
        m = [] 
        s = -10000
        max = -100000
        for i in range(len(arr)): 
            a = arr[i] 
            if a[0] > max: 
                if i != 0: 
                    m.append([s,max]) 
                max = a[1] 
                s = a[0] 
            else: 
                if a[1] >= max: 
                    max = a[1] 
          
        #'max' value gives the last point of  
        # that particular interval 
        # 's' gives the starting point of that interval 
        # 'm' array contains the list of all merged intervals 
  
        if max != -100000 and [s, max] not in m: 
            m.append([s, max]) 
        #print("The Merged Intervals are :", end = " ") 
        #for i in range(len(m)): 
            #print(m[i], end = " ") 
        return m

In [ ]:
dialog_silences = {}
dialog_transitions = {}
for i in range(0,6):
    df = dfs_speaker_event[i][["frame_start","frame_end"]]
    last_frame = int(dialog_durations.iloc[:, [i]].values[0]/40)
    lst = [[1,1]] + df.to_numpy().tolist() + [[last_frame,last_frame]]
    non_overlapping_lst = mergeIntervals(lst)
    #find transition intervals
    lst = non_overlapping_lst
    silences = [(lst[j][1], lst[j+1][0]) for j in range(len(lst)-1) if lst[j][1] < lst[j+1][0]]
    dialog_silences[i] = silences
    dialog_transitions[i] = silences[1:-1]

In [ ]:
dialog_transitions

In [ ]:
silences[1:-1]

Let's observe the distribution of the durations of transitions in each dialog :

In [ ]:
for i in range(0,6):
    plt.figure(figsize=(12,8))
    transitions_dur = [j[1]-j[0] for j in dialog_transitions[i]]
    plt.title('Transitions in dialog'+str(i+1))
    plt.hist(transitions_dur)
    plt.xlabel('Durations in nb of frame')
    plt.ylabel('Nb of transitions')
    plt.show()
    
    c = len([j for j in transitions_dur if j < 8])
    print('Total nb of transitions: '+str(len(dialog_transitions[i])))
    print('Total nb of transitions below 8 frames: '+str(c))

We can see that the turn transitions are mostly very fast in each dialog (between 40 to 320 ms). So we can separate fast turn transitions (durations of 320 ms or less) and normal turn transitions (+320ms)

In [ ]:
dialog_fast_transitions = {}
dialog_normal_transitions = {}
for i in range(0,6):
    fast_transitions = [(j[0],j[1]) for j in dialog_transitions[i] if (j[1]-j[0])<=8]
    normal_transitions = [(j[0],j[1]) for j in dialog_transitions[i] if (j[1]-j[0])>8]
    dialog_normal_transitions[i] = normal_transitions
    dialog_fast_transitions[i] = fast_transitions

In [ ]:
#dialog_transitions[0]

In [ ]:
#dialog_normal_transitions[0]

In [ ]:
#dialog_fast_transitions[0]

To analyze the transitions I construct a window of 2 sec (+-25 frames from the center) for the fast transition and a window of 5 sec (+-63 frames from the center) for the normal transition

In [ ]:
half_fast_window = 25
half_normal_window = 63
analysis_windows_fast = {}
analysis_windows_normal = {}

for i in range(0,6):
    analysis_windows_fast[i]=[]
    analysis_windows_normal[i]=[]
    fast_trans = dialog_fast_transitions[i]
    normal_trans = dialog_normal_transitions[i]
    for trans in fast_trans:
        start = round((trans[0]+trans[1])/2)-half_fast_window
        end = round((trans[0]+trans[1])/2)+half_fast_window
        analysis_windows_fast[i].append(tuple([start,end]))
    for trans in normal_trans:
        start = round((trans[0]+trans[1])/2)-half_normal_window
        end = round((trans[0]+trans[1])/2)+half_normal_window
        analysis_windows_normal[i].append(tuple([start,end]))

In [ ]:
#analysis_windows_normal[0]

In [ ]:
analysis_windows_fast[0]

### Evolution of fixation dispersion at turn transitions

#### Fast transitions

In [ ]:
mean_fast_transitions_disp = {}
for dialog_nb in range(0,6):
    mean_fast_transitions_disp[dialog_nb] = []
    for window_nb, window in enumerate(analysis_windows_fast[dialog_nb]):
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
            frame_end = len(list_points_dialog_disp[dialog_nb])-1
        plt.figure(figsize=(12,8))
        plt.title('Dispersion at turn transition (window frames: '+str(frame_start)+' to '+str(frame_end)+
                  ') (transition frames: '+str(dialog_fast_transitions[dialog_nb][window_nb][0])+' to '+str(dialog_fast_transitions[dialog_nb][window_nb][1])+') in dialog '+str(dialog_nb+1)
                 + '. Silence is marked in orange.'
                 )
        
        
        x = [point[0]/40 for point in list_points_dialog_disp[dialog_nb][frame_start:frame_end]]
        y = [point[1] for point in list_points_dialog_disp[dialog_nb][frame_start:frame_end]]
        mean_fast_transitions_disp[dialog_nb].append(np.mean(y))
        plt.plot(x, y, marker='x', color='cornflowerblue')
        plt.xlabel('Frames')
        plt.ylabel('Dispersion')
        #plt.legend()
        plt.axvspan(dialog_fast_transitions[dialog_nb][window_nb][0], dialog_fast_transitions[dialog_nb][window_nb][1], color='bisque', alpha=0.4) ## highlight silent area
        
        plt.show()
        #plt.savefig('dispersion.png')

Export to CSV

In [ ]:
dialog_nb = 0
for item_name in fixations_original_all.index.unique():
    df = pd.DataFrame(mean_fast_transitions_disp[dialog_nb],columns=[item_name])
    df.to_csv('./results/transitions/fast-new/'+item_name+'_mean_dispersion_transitions_fast_per_frame.csv', index=False)
    dialog_nb = dialog_nb+1

In [ ]:
mean_dispersion_fast_transitions = {}
for dialog in mean_fast_transitions_disp:
    mean_dispersion_fast_transitions[dialog+1]=np.mean(mean_fast_transitions_disp[dialog])

In [ ]:

mean_dispersion_fast_transitions    

#### Normal transitions

In [ ]:
mean_normal_transitions_disp = {}
for dialog_nb in range(0,6):
    mean_normal_transitions_disp[dialog_nb] = []
    for window_nb, window in enumerate(analysis_windows_normal[dialog_nb]):
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
            frame_end = len(list_points_dialog_disp[dialog_nb])-1
        plt.figure(figsize=(12,8))
        plt.title('Dispersion at turn transition (window frames: '+str(frame_start)+' to '+str(frame_end)+
                  ') (transition frames: '+str(dialog_normal_transitions[dialog_nb][window_nb][0])+' to '+str(dialog_normal_transitions[dialog_nb][window_nb][1])+') in dialog '+str(dialog_nb+1)
                  + '. Silence is marked in orange.'   
                 )
        x = [point[0]/40 for point in list_points_dialog_disp[dialog_nb][frame_start:frame_end]]
        y = [point[1] for point in list_points_dialog_disp[dialog_nb][frame_start:frame_end]]
        
        mean_normal_transitions_disp[dialog_nb].append(np.mean(y))
        plt.plot(x, y, marker='x', color='cornflowerblue')
        plt.xlabel('Frames')
        plt.ylabel('Dispersion')
        
        plt.axvspan(dialog_normal_transitions[dialog_nb][window_nb][0], dialog_normal_transitions[dialog_nb][window_nb][1], color='bisque', alpha=0.4) ## highlight silent area
        #plt.legend()
        plt.show()
        #plt.savefig('dispersion.png')

Export to CSV

In [ ]:
dialog_nb = 0
for item_name in fixations_original_all.index.unique():
    df = pd.DataFrame(mean_normal_transitions_disp[dialog_nb],columns=[item_name])
    df.to_csv('./results/transitions/normal-new/' +item_name+'_mean_dispersion_transitions_normal_per_frame.csv', index=False)
    dialog_nb = dialog_nb+1

In [ ]:
mean_dispersion_normal_transitions = {}
for dialog in mean_normal_transitions_disp:
    mean_dispersion_normal_transitions[dialog+1]=np.mean(mean_normal_transitions_disp[dialog])

In [ ]:
mean_dispersion_normal_transitions   

In [ ]:
mean_dispersion_normal_transitions   

### Evolution of aoi fixation ratios at turn transitions

#### Fast

In [ ]:
list(aoi_ratios_per_frame.keys())[0]

In [ ]:
areas_of_interest = ['L','R','F','e']
aoi_ratio_fast_transitions = {}
for dialog_nb in range(0,6):
    aoi_ratio_fast_transitions[dialog_nb] = {}
    for window_nb, window in enumerate(analysis_windows_fast[dialog_nb]):
        aoi_ratio_fast_transitions[dialog_nb][window_nb] = {}
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
            frame_end = len(list_points_dialog_disp[dialog_nb])-1
        plt.figure(figsize=(20,4))
        plt.title('Fixation ratios on AoIs over frames for dialog '+str(dialog_nb+1)+' (window frames: '+str(frame_start)+' to '+str(frame_end)+') (transition frames: '+str(dialog_fast_transitions[dialog_nb][window_nb][0])+' to '+str(dialog_fast_transitions[dialog_nb][window_nb][1])+')'
                 + '. Silence is marked in orange.' 
                 )
        for i in areas_of_interest:
            aoi_ratio_fast_transitions[dialog_nb][window_nb][i] = []
            x = []
            y = []
            for frame in range(frame_start-1, frame_end):
                x.append(frame)
                y.append(aoi_ratios_per_frame[list(aoi_ratios_per_frame.keys())[dialog_nb]][frame][i])
                aoi_ratio_fast_transitions[dialog_nb][window_nb][i].append(y)
            if i == 'L':
                plt.plot(x, y, label='left', marker='o', color='r')
            elif i == 'R':
                plt.plot(x, y, label='right', marker='x', color='g')
            elif i == 'F':
                plt.plot(x, y, label='front', marker='^', color='orange')
            else:
                plt.plot(x, y, label='exterior', marker='+', color='gray')
                
        plt.axvspan(dialog_fast_transitions[dialog_nb][window_nb][0], dialog_fast_transitions[dialog_nb][window_nb][1], color='bisque', alpha=0.4) ## highlight silent area       
        
        plt.xlabel('Frame')
        plt.ylabel('Fixation ratio')
        plt.legend()
        plt.show()
        #plt.savefig('dispersion.png')

#### Normal

In [ ]:
areas_of_interest = ['L','R','F','e']
aoi_ratio_normal_transitions = {}
for dialog_nb in range(0,6):
    aoi_ratio_normal_transitions[dialog_nb] = {}
    for window_nb, window in enumerate(analysis_windows_normal[dialog_nb]):
        try:
            aoi_ratio_normal_transitions[dialog_nb][window_nb] = {}
            frame_start = window[0]
            frame_end = window[1]
            if frame_start < 0:
                frame_start = 0
            if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
                frame_end = len(list_points_dialog_disp[dialog_nb])-1
            plt.figure(figsize=(20,4))
            plt.title('Fixation ratios on AoIs over frames for dialog '+str(dialog_nb+1)+' (window frames: '+str(frame_start)+' to '+str(frame_end)+') (transition frames: '+str(dialog_normal_transitions[dialog_nb][window_nb][0])+' to '+str(dialog_normal_transitions[dialog_nb][window_nb][1])+')'
                     + '. Silence is marked in orange.' 
                     )
            for i in areas_of_interest:
                aoi_ratio_normal_transitions[dialog_nb][window_nb][i] = []
                x = []
                y = []
                for frame in range(frame_start-1, frame_end):
                    x.append(frame)
                    y.append(aoi_ratios_per_frame[list(aoi_ratios_per_frame.keys())[dialog_nb]][frame][i])
                    aoi_ratio_normal_transitions[dialog_nb][window_nb][i].append(y)
                if i == 'L':
                    plt.plot(x, y, label='left', marker='o', color='r')
                elif i == 'R':
                    plt.plot(x, y, label='right', marker='x', color='g')
                elif i == 'F':
                    plt.plot(x, y, label='front', marker='^', color='orange')
                else:
                    plt.plot(x, y, label='exterior', marker='+', color='gray')

            plt.axvspan(dialog_normal_transitions[dialog_nb][window_nb][0], dialog_normal_transitions[dialog_nb][window_nb][1], color='bisque', alpha=0.4) ## highlight silent area       

            plt.xlabel('Frame')
            plt.ylabel('Fixation ratio')
            plt.legend()
            plt.show()
        except:
            print('Fixation ratios on AoIs over frames for dialog '+str(dialog_nb+1)+' (window frames: '+str(frame_start)+' to '+str(frame_end)+') (transition frames: '+str(dialog_normal_transitions[dialog_nb][window_nb][0])+' to '+str(dialog_normal_transitions[dialog_nb][window_nb][1])+')'
                     + ' cannot be shown.')
        #plt.savefig('dispersion.png')

In [ ]:
mean_aoi_ratio_fast_transitions = {}
for dialog in aoi_ratio_fast_transitions:
    mean_aoi_ratio_fast_transitions[dialog] = {}
    for window in aoi_ratio_fast_transitions[dialog]:
        mean_aoi_ratio_fast_transitions[dialog][window] = {}
        for aoi in aoi_ratio_fast_transitions[dialog][window]:
            mean_aoi_ratio_fast_transitions[dialog][window][aoi] = np.mean(aoi_ratio_fast_transitions[dialog][window][aoi])
            
mean_aoi_ratio_fast_transitions

### Evolution of distance to center at turn transitions

In [ ]:
#list_points_dialog_center

In [ ]:
for dialog_nb in range(0,6):
    for window_nb, window in enumerate(analysis_windows_fast[dialog_nb]):
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_center[dialog_nb])-1:
            frame_end = len(list_points_dialog_center[dialog_nb])-1
        plt.figure(figsize=(12,8))
        plt.title('Distance to center at turn transition (window: '+str(frame_start)+' to '+str(frame_end)+
                  ') (transition: '+str(dialog_fast_transitions[dialog_nb][window_nb][0])+' to '+str(dialog_fast_transitions[dialog_nb][window_nb][1])+') in dialog '+str(dialog_nb+1))
        x = [point[0]/40 for point in list_points_dialog_center[dialog_nb][frame_start:frame_end]]
        y = [point[1] for point in list_points_dialog_center[dialog_nb][frame_start:frame_end]]
        plt.plot(x, y, marker='x', color='r')
        plt.xlabel('Frames')
        plt.ylabel('Distance to center')
        #plt.legend()
        plt.show()

### Evolution of scanpaths similarity at turn transitions (also patterns? transition probability?)

Step1: visually look at speech act v.s. genetic scanpath at turning points

Step2: 3D plot of fixation of each participant during transition period

In [ ]:
# check if there are more than one fixation in a frame
for dialog_nb, dialog in enumerate(fixation_points_per_participant_per_frame):
    print(dialog_nb)
    for i in range (0,len(dialog)):
            for frame_nb in range(0, len(dialog[i])):  
                if len(dialog[i][frame_nb]) > 1:
                    print("dialog: " + str(dialog_nb))
                    print("participant: " + str(i))
                    print("frame: " + str(frame_nb))
                    print(dialog[i][frame_nb])
                    print("--------------------")

In [ ]:
# example two fixations in one frame
dialog[41][2918]

In [ ]:
# eample one fixation in one frame
dialog[41][2919]

Just take the first fixation if there are more than one fixations in a frame as the value does not change significantly 

In [ ]:
for dialog_nb, dialog in enumerate(fixation_points_per_participant_per_frame):
    
    # fast transition
    for window_nb, window in enumerate(analysis_windows_fast[dialog_nb]):
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
            frame_end = len(list_points_dialog_disp[dialog_nb])-1
        
        
        fig = plt.figure(figsize = (12, 8))
        ax = plt.axes(projection ="3d")
        plt.title("3D representation of scanpath of each participant in dialog "+str(dialog_nb+1) + 
                  " during fast transitions" + " (window frames: " +str(frame_start) + " to " +str(frame_end)+ ")" ) 
        
        # plot image
        img = read_png('./screenshot/dialog' + str(dialog_nb+1) + '_screenshot.png')
        x, y = ogrid[0:img.shape[0], 0:img.shape[1]]
        # ax = gca(projection='3d')
        ax.plot_surface(x, y, np.atleast_2d(frame_start), rstride=8, cstride=8, facecolors=img, alpha=0.2)

        for i in range (0,42): # loop through 42 participantw
            participant = dialog[i]
            x = []
            y = []
            z = []
            
            for frame_nb in range(frame_start-1, frame_end):
                if participant[frame_nb] != []:
                    x.append(participant[frame_nb][0][1])
                    y.append(participant[frame_nb][0][0])
                    z.append(frame_nb)

            ax.set_xlim(0,720)
            ax.set_ylim(0,1280)
            ax.set_zlim(frame_start,frame_end)

            ax.set_xlabel("y")
            ax.set_ylabel("x")
            ax.set_zlabel("frames")
            ax.plot3D(np.array(x), np.array(y), np.array(z))
        ax.view_init(elev=25, azim=-15)
        plt.savefig('./scanpath_plot/fast_transition/dialog'+ str(dialog_nb+1)+'/' + 'window_start_'+str(frame_start)+'.png')

In [ ]:
for dialog_nb, dialog in enumerate(fixation_points_per_participant_per_frame):
    
    # fast transition
    for window_nb, window in enumerate(analysis_windows_normal[dialog_nb]):
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
            frame_end = len(list_points_dialog_disp[dialog_nb])-1
        
        
        fig = plt.figure(figsize = (12, 8))
        ax = plt.axes(projection ="3d")
        plt.title("3D representation of scanpath of each participant in dialog "+str(dialog_nb+1) + 
                  " during normal transitions" + " (window frames: " +str(frame_start) + " to " +str(frame_end)+ ")" ) 
        
        # plot image
        img = read_png('./screenshot/dialog' + str(dialog_nb+1) + '_screenshot.png')
        x, y = ogrid[0:img.shape[0], 0:img.shape[1]]
        # ax = gca(projection='3d')
        ax.plot_surface(x, y, np.atleast_2d(frame_start), rstride=8, cstride=8, facecolors=img, alpha=0.2)

        for i in range (0,42): # loop through 42 participantw
            participant = dialog[i]
            x = []
            y = []
            z = []
            
            for frame_nb in range(frame_start-1, frame_end):
                if participant[frame_nb] != []:
                    x.append(participant[frame_nb][0][1])
                    y.append(participant[frame_nb][0][0])
                    z.append(frame_nb)

            ax.set_xlim(0,720)
            ax.set_ylim(0,1280)
            ax.set_zlim(frame_start,frame_end)

            ax.set_xlabel("y")
            ax.set_ylabel("x")
            ax.set_zlabel("frames")
            ax.plot3D(np.array(x), np.array(y), np.array(z))
        ax.view_init(elev=25, azim=-15)

### Multiple speaker event (sense of interruption/agreement)

In [ ]:
half_fast_window = 25
half_normal_window = 63
analysis_windows_fast = {}
analysis_windows_normal = {}

for i in range(0,6):
    analysis_windows_fast[i]=[]
    analysis_windows_normal[i]=[]
    fast_trans = dialog_fast_transitions[i]
    normal_trans = dialog_normal_transitions[i]
    for trans in fast_trans:
        start = round((trans[0]+trans[1])/2)-half_fast_window
        end = round((trans[0]+trans[1])/2)+half_fast_window
        analysis_windows_fast[i].append(tuple([start,end]))
    for trans in normal_trans:
        start = round((trans[0]+trans[1])/2)-half_normal_window
        end = round((trans[0]+trans[1])/2)+half_normal_window
        analysis_windows_normal[i].append(tuple([start,end]))

In [ ]:
tuple([start,end])

In [ ]:
interruptions = {}

# half_fast_window = 25
# half_normal_window = 63
# analysis_windows_fast = {}
# analysis_windows_normal = {}

# for i in range(0,6):
#     analysis_windows_fast[i]=[]
#     analysis_windows_normal[i]=[]
#     fast_trans = dialog_fast_transitions[i]
#     normal_trans = dialog_normal_transitions[i]
#     for trans in fast_trans:
#         start = round((trans[0]+trans[1])/2)-half_fast_window
#         end = round((trans[0]+trans[1])/2)+half_fast_window
#         analysis_windows_fast[i].append(tuple([start,end]))
#     for trans in normal_trans:
#         start = round((trans[0]+trans[1])/2)-half_normal_window
#         end = round((trans[0]+trans[1])/2)+half_normal_window
#         analysis_windows_normal[i].append(tuple([start,end]))

    
    
for dialog_nb in range(0,6):
    interruptions[dialog_nb]=[]
    df = dfs_speaker_event[dialog_nb][["frame_start","frame_end"]]
    df = df.sort_values(by=['frame_start'])
    
    for i in range(0, df.shape[0]): # iterate rows
        df_prev = df.iloc[:i]
        for j in range(0, df_prev.shape[0]):
            if df.iloc[i]['frame_end'] < df_prev.iloc[j]['frame_end']:
                interr = tuple([df.iloc[i]['frame_start'],df.iloc[i]['frame_end']]) 
                interruptions[dialog_nb].append(interr)



In [ ]:
for i in range (0, len(interruptions)):
    print(len(interruptions[i]))

### Evolution of fixation dispersion at interruptions

create a window of +-2 sec (25 frames) for the interruption

In [ ]:
half_fast_window = 25
analysis_interruptions = {}

for i in range(0,6):
    analysis_interruptions[i]=[]

    interr = interruptions[i]
    normal_trans = dialog_normal_transitions[i]
    for trans in interr:
        start = round((trans[0]+trans[1])/2)-half_fast_window
        end = round((trans[0]+trans[1])/2)+half_fast_window
        analysis_interruptions[i].append(tuple([int(start),int(end)]))

In [ ]:
mean_interruption_disp = {}
for dialog_nb in range(0,6):
    mean_interruption_disp[dialog_nb] = []
    for window_nb, window in enumerate(analysis_interruptions[dialog_nb]):
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
            frame_end = len(list_points_dialog_disp[dialog_nb])-1
        plt.figure(figsize=(12,8))
        plt.title('Dispersion at interruption (window frames: '+str(frame_start)+' to '+str(frame_end)+
                  ') (transition frames: '+str(interruptions[dialog_nb][window_nb][0])+' to '+str(interruptions[dialog_nb][window_nb][1])+') in dialog '+str(dialog_nb+1)
                 + '. Interruption is marked in pink.'
                 )
        
        
        x = [point[0]/40 for point in list_points_dialog_disp[dialog_nb][frame_start:frame_end]]
        y = [point[1] for point in list_points_dialog_disp[dialog_nb][frame_start:frame_end]]
        mean_interruption_disp[dialog_nb].append(np.mean(y))
        plt.plot(x, y, marker='x', color='cornflowerblue')
        plt.xlabel('Frames')
        plt.ylabel('Dispersion')
        #plt.legend()
        plt.axvspan(interruptions[dialog_nb][window_nb][0], interruptions[dialog_nb][window_nb][1], color='pink', alpha=0.3) ## highlight silent area
        
        plt.show()

export to csv

In [ ]:
dialog_nb = 0
for item_name in fixations_original_all.index.unique():
    df = pd.DataFrame(mean_interruption_disp[dialog_nb],columns=[item_name])
    df.to_csv('./results/interruptions/dispersion/' +item_name+'_mean_dispersion_interruption_per_frame.csv', index=False)
    dialog_nb = dialog_nb+1

In [ ]:
mean_dispersion_interruption = {}
for dialog in mean_interruption_disp:
    mean_dispersion_interruption[dialog+1]=np.mean(mean_interruption_disp[dialog])

In [ ]:
mean_dispersion_interruption

### Evolution of AOI fixation ratio at turn interruptions

In [ ]:
areas_of_interest = ['L','R','F','e']
aoi_ratio_interruptions = {}
for dialog_nb in range(0,6):
    aoi_ratio_interruptions[dialog_nb] = {}
    for window_nb, window in enumerate(analysis_interruptions[dialog_nb]):
        aoi_ratio_interruptions[dialog_nb][window_nb] = {}
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
            frame_end = len(list_points_dialog_disp[dialog_nb])-1
        plt.figure(figsize=(20,4))
        plt.title('Fixation ratios on AoIs over frames for dialog '+str(dialog_nb+1)+' (window frames: '+str(frame_start)+' to '+str(frame_end)+') (transition frames: '+str(interruptions[dialog_nb][window_nb][0])+' to '+str(interruptions[dialog_nb][window_nb][1])+')'
                 + '. Silence is marked in pink.' 
                 )
        for i in areas_of_interest:
            aoi_ratio_interruptions[dialog_nb][window_nb][i] = []
            x = []
            y = []
            for frame in range(frame_start-1, frame_end):
                x.append(frame)
                y.append(aoi_ratios_per_frame[list(aoi_ratios_per_frame.keys())[dialog_nb]][frame][i])
                aoi_ratio_interruptions[dialog_nb][window_nb][i].append(y)
            if i == 'L':
                plt.plot(x, y, label='left', marker='o', color='r')
            elif i == 'R':
                plt.plot(x, y, label='right', marker='x', color='g')
            elif i == 'F':
                plt.plot(x, y, label='front', marker='^', color='orange')
            else:
                plt.plot(x, y, label='exterior', marker='+', color='gray')
                
        plt.axvspan(interruptions[dialog_nb][window_nb][0], interruptions[dialog_nb][window_nb][1], color='pink', alpha=0.3) ## highlight silent area       
        
        plt.xlabel('Frame')
        plt.ylabel('Fixation ratio')
        plt.legend()
        plt.show()
        #plt.savefig('dispersion.png')

### Evolution of scanpaths similarity at interruptions

In [ ]:
for dialog_nb, dialog in enumerate(fixation_points_per_participant_per_frame):
    
    # fast transition
    for window_nb, window in enumerate(analysis_interruptions[dialog_nb]):
        frame_start = window[0]
        frame_end = window[1]
        if frame_start < 0:
            frame_start = 0
        if frame_end > len(list_points_dialog_disp[dialog_nb])-1:
            frame_end = len(list_points_dialog_disp[dialog_nb])-1
        
        
        fig = plt.figure(figsize = (12, 8))
        ax = plt.axes(projection ="3d")
        plt.title("3D representation of scanpath of each participant in dialog "+str(dialog_nb+1) + " during interruptions" + " (window frames: " +str(frame_start) + " to " +str(frame_end)+ ")" ) 
        
        # plot image
        # fn = get_sample_data("dialog5_screenshot.png", asfileobj=False)
        img = read_png('./screenshot/dialog' + str(dialog_nb+1) + '_screenshot.png')
        x, y = ogrid[0:img.shape[0], 0:img.shape[1]]
        # ax = gca(projection='3d')
        ax.plot_surface(x, y, np.atleast_2d(frame_start), rstride=8, cstride=8, facecolors=img, alpha=0.2)

        for i in range (0,42): # loop through 42 participantw
            participant = dialog[i]
            x = []
            y = []
            z = []
            
            for frame_nb in range(frame_start-1, frame_end):
                if participant[frame_nb] != []:
                    x.append(participant[frame_nb][0][1])
                    y.append(participant[frame_nb][0][0])
                    z.append(frame_nb)

            ax.set_xlim(0,720)
            ax.set_ylim(0,1280)
            ax.set_zlim(frame_start,frame_end)

            ax.set_xlabel("y")
            ax.set_ylabel("x")
            ax.set_zlabel("frames")
            ax.plot3D(np.array(x), np.array(y), np.array(z))
        ax.view_init(elev=25, azim=-15)
        plt.savefig('./scanpath_plot/interruptions/dialog'+ str(dialog_nb+1)+'/' + 'window_start_'+str(frame_start)+'.png')
        
        